In [ ]:
"""HES Kodları üretim sistemi.
HES Kodlarının toplu tutulup kontrol yapılan yerlerde gerekli olabilir.

Aşağıdaki kütüphaneleri kurmak gerekli olacaktır.
!pip install pyqrcode
!pip install pypng
!pip install weasyprint
"""

In [ ]:
# -*- coding: utf-8 -*-
import locale # Listelemek için bunu kullandım ama bi boka yaramadı.
locale.setlocale(locale.LC_ALL, 'tr_TR.UTF-8')
import pandas as pd
import pyqrcode 
import png 

In [ ]:
df = pd.read_csv('HES_list.csv', header=0)
df['cleanQR'] = df['qrCode'].str.replace(r'\W+', '')
df.drop_duplicates(subset ="cleanQR", keep = 'last', inplace = True)
df = df.sort_values(by=['Name'])
#print(df)

In [ ]:
# Create html file:
html = """
<!DOCTYPE html>
<html>
<meta charset="utf-8"/>
<head>
<title>HES Kodları Listesi</title>
<style>
.floating-box {
  float: left;
  width: 120px;
  height: 130px;
  margin: 2px;
  border: 3px solid #73AD21;
  text-align: center;
  display: block;
}
span.small {
  display: inline-block;
  font-size: smaller;
}
</style>
</head>
<body>
<h2>HES Kodları Listesi</h2>
"""

In [ ]:
from io import BytesIO
import base64

def pngMaker(text):   
    qrData = text
    image = pyqrcode.create(qrData)
    buffer = BytesIO()
    image.png(buffer, scale = 3)
    img_str = base64.b64encode(buffer.getvalue()).decode("utf-8")
   
    return img_str

In [ ]:
for ind, value in df.iterrows():
    text = '<span class="small">'+value['Name']+'<br>'+value['qrCode']+'</span>'
    html += '<div class="floating-box"><img src="data:image/png;base64,'+pngMaker(value['cleanQR'])+'">'+text+'</div>'+"\n"
    
html += '</body></html>'

In [ ]:
# save html file with images embedded
# with open("index.html", "w") as outf:
#     outf.write(str(html))

In [ ]:
from weasyprint import HTML, CSS

htmlpdf = HTML(string=html) 
css = CSS(string='@page {size: A4 portrait; margin: 1cm;}')
htmlpdf.write_pdf('pdfoutput.pdf', stylesheets=[css])